In [1]:
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
imageSize = [299, 299]

trainPath = r"/content/drive/MyDrive/Covid_dataset/train"

testPath = r"/content/drive/MyDrive/Covid_dataset/test"

In [4]:
inception = InceptionV3(input_shape=imageSize + [3], weights='imagenet',include_top=False)

87924736/87910968 [==============================] - 0s 0us/step


In [5]:
# don't train existing weights
for layer in inception.layers:
  layer.trainable = False

In [6]:
# our layers - you can add more if you want
x = Flatten()(inception.output)

In [7]:
prediction = Dense(4, activation='softmax')(x)

In [8]:
# create a model object
model = Model(inputs=inception.input, outputs=prediction)

In [9]:
# view the structure of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 149, 149, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 149, 149, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [10]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


In [11]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [12]:
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Covid_dataset/train',
                                                 target_size = (299, 299),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Covid_dataset/test',
                                            target_size = (299, 299),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 19159 images belonging to 4 classes.
Found 2006 images belonging to 4 classes.


In [13]:
# fit the model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=25,
  steps_per_epoch=len(training_set)//32,
  validation_steps=len(test_set)//32
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  import sys


Epoch 1/25
18/18 [==============================] - 189s 10s/step - loss: 16.4655 - accuracy: 0.5122 - val_loss: 7.9457 - val_accuracy: 0.6250
Epoch 2/25
18/18 [==============================] - 124s 7s/step - loss: 7.5991 - accuracy: 0.6597 - val_loss: 14.0341 - val_accuracy: 0.5000
Epoch 3/25
18/18 [==============================] - 119s 7s/step - loss: 4.9129 - accuracy: 0.7569 - val_loss: 3.7073 - val_accuracy: 0.6250
Epoch 4/25
18/18 [==============================] - 118s 7s/step - loss: 4.1372 - accuracy: 0.7274 - val_loss: 2.5814 - val_accuracy: 0.8125
Epoch 5/25
18/18 [==============================] - 112s 6s/step - loss: 2.8889 - accuracy: 0.7795 - val_loss: 2.5765 - val_accuracy: 0.7812
Epoch 6/25
18/18 [==============================] - 113s 6s/step - loss: 2.2420 - accuracy: 0.7882 - val_loss: 6.3533 - val_accuracy: 0.6875
Epoch 7/25
18/18 [==============================] - 105s 6s/step - loss: 2.8446 - accuracy: 0.7552 - val_loss: 6.8846 - val_accuracy: 0.6562
Epoch 8/25

In [14]:
model.save('InceptionV3-covid.h5')

In [15]:
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.applications.inception_v3 import decode_predictions

In [16]:
img = '/content/drive/MyDrive/Covid_dataset/train/COVID/COVID-100.png'
image = load_img(img, target_size=(299,299))
image = img_to_array(image)
image = image.reshape((1,299,299,3))

In [17]:
image.shape

(1, 299, 299, 3)

In [18]:
model.predict(image)

array([[1., 0., 0., 0.]], dtype=float32)